In [1]:
%load_ext Cython

### MCTS Testing

#### Plain Python

In [3]:
%%cython
# cython: profile=True
import pstats, cProfile

from game import Game
from config import Config
import chess
import numpy as np
cimport numpy as np
import tensorflow as tf
from mcts import run_mcts

def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    value = tf.convert_to_tensor([[value]], dtype=tf.float32)
    policy = tf.convert_to_tensor(policy, dtype=tf.float32)
    return {
        "value_head": value,
        "policy_head": [policy],
    }

def test():
    board = chess.Board()
    game = Game(board)
    config = Config()
    moves = ["e2e4", "e7e5", "g1f3", "b8c6", "f1c4", "g8f6", "d2d3", "f8b4", "c1d2"]
    for move in moves:
        game.make_move(move)
    for i in range(100):
        move, root = run_mcts(
            config,
            game,
            fake_network,
            None,
            False,
            600,
            30,
        )

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Content of stderr:
In file included from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/tomaz/.cache/ipython/cython/_cython_magic_485f1ac55288963485a2b8a049d0b3d9fc77d962.c:1216:
/home/tomaz/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

2024-02-17 12:54:31.272390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 12:54:32.677364: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5922 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5


Sat Feb 17 12:56:46 2024    Profile.prof

         217823243 function calls (217210327 primitive calls) in 132.773 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2354708    9.578    0.000   24.308    0.000 __init__.py:2200(push)
    60101    8.669    0.000    9.018    0.000 constant_op.py:65(convert_to_eager_tensor)
 12583190    4.592    0.000    5.170    0.000 __init__.py:314(scan_reversed)
  4595762    4.011    0.000   10.883    0.000 __init__.py:1706(generate_pseudo_legal_moves)
   120308    3.884    0.000    3.884    0.000 {built-in method numpy.array}
    60100    3.802    0.000   16.004    0.000 __init__.py:125(expand)
  4821031    3.778    0.000    5.109    0.000 __init__.py:901(_remove_piece_at)
  5779562    3.760    0.000    4.367    0.000 __init__.py:168(ucb)
    60100    3.508    0.000    3.508    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_Execute}
    60000    2.970    0.000   40.988    0.001

#### C++ & Cython

In [2]:
%%cython
# cython: profile=True
import pstats, cProfile

from game import Game
from config import Config
import cppchess as chess
import numpy as np
cimport numpy as np
import tensorflow as tf
from mcts.c import run_mcts

def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    value = tf.convert_to_tensor([[value]], dtype=tf.float32)
    policy = tf.convert_to_tensor(policy, dtype=tf.float32)
    return {
        "value_head": value,
        "policy_head": [policy],
    }

board = chess.Board()
game = Game(board)
config = Config()
moves = ["e2e4", "e7e5", "g1f3", "b8c6", "f1c4", "g8f6", "d2d3", "f8b4", "c1d2"]
for move in moves:
    game.make_move(move)

def test():
    for i in range(100):
        move, root = run_mcts(
            config,
            game,
            fake_network,
            None,
            False,
            600,
            30,
        )

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

2024-02-17 13:56:34.673425: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 13:56:36.011391: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5829 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5


Sat Feb 17 13:57:27 2024    Profile.prof

         48142539 function calls (47640230 primitive calls) in 51.573 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   13.567   13.567   51.617   51.617 _cython_magic_de6c23651271d3cc157737644ff656d75c55a7e4.pyx:29(test)
    60101    7.376    0.000    7.749    0.000 constant_op.py:65(convert_to_eager_tensor)
    60100    3.172    0.000    3.172    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_Execute}
   240438    1.489    0.000    2.101    0.000 typing.py:1408(_get_protocol_attrs)
   180303    1.378    0.000    2.325    0.000 inspect.py:3050(_bind)
3044896/3044873    0.811    0.000    4.387    0.000 {built-in method builtins.isinstance}
    60100    0.682    0.000    5.319    0.000 atomic_function.py:163(__call__)
    60100    0.653    0.000   35.357    0.001 polymorphic_function.py:797(__call__)
    60101    0.649    0.000    8.800    0.000 function_type.p

In [1]:
%load_ext Cython

### Image generation & update testing

#### Plain python

In [3]:
%%cython
# cython: profile=True
import pstats, cProfile
from gameimage import board_to_image, update_image
import chess

def test():
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")
    board.push_san("a3")
    board.push_san("d5")
    board.push_san("Nc3")
    board.push_san("Nc6")
    #image = board_to_image(board)
    for i in range(10001):
        image = board_to_image(board)
        #update_image(board, image)

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Sat Feb 17 12:47:55 2024    Profile.prof

         21072454 function calls in 9.190 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    70007    1.999    0.000    2.045    0.000 __init__.py:22(pieces_onehot)
  2240224    1.129    0.000    2.125    0.000 __init__.py:725(piece_at)
    70007    1.050    0.000    4.109    0.000 __init__.py:1075(piece_map)
    70007    0.982    0.000    2.346    0.000 __init__.py:10(parse_piece_map)
  2310277    0.649    0.000    0.779    0.000 __init__.py:314(scan_reversed)
  2240242    0.642    0.000    0.642    0.000 __init__.py:735(piece_type_at)
  2240224    0.611    0.000    1.322    0.000 __init__.py:496(symbol)
  2240224    0.491    0.000    0.636    0.000 __init__.py:60(piece_symbol)
    10001    0.419    0.000    9.537    0.001 __init__.py:33(board_to_image)
  4480448    0.300    0.000    0.300    0.000 typing.py:1737(cast)
  2240244    0.129    0.000    0.129    0.000 {method 'bit_len

#### C++ and Cython

In [4]:
%%cython
# cython: profile=True
import pstats, cProfile
from gameimage.c cimport board_to_image, update_image
import cppchess as chess

def test():
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")
    board.push_san("a3")
    board.push_san("d5")
    board.push_san("Nc3")
    board.push_san("Nc6")
    #image = board_to_image(board)
    for i in range(10000):
        image = board_to_image(board)
        #update_image(board, image)

cProfile.runctx("test()", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Fri Feb 23 12:47:32 2024    Profile.prof

         4 function calls in 1.133 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.133    1.133    1.133    1.133 _cython_magic_2f1e6f3e7b6c5e4ed8568cec2dc7626749cda068.pyx:6(test)
        1    0.000    0.000    1.133    1.133 {built-in method builtins.exec}
        1    0.000    0.000    1.133    1.133 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [2]:
from game import Game
import cppchess as chess
from multiprocessing import Process
from mcts.n import run_mcts
from config import Config
from time import time

def play_game(id):
    import tensorflow as tf
    gpu_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

    from tf_funcs import fake_network
    config = Config()
    for i in range(2):
        game = Game(chess.Board())
        time1 = time()
        while not game.terminal_with_outcome():
            move, _ = run_mcts(
                config,
                game,
                fake_network,
                None,
                False,
                300,
                30,
            )
            game.make_move(move)
        time2 = time()
        print(f"{id}: Game {i} finished in: {time2 - time1}. Moves: {game.history_len}. Time per move: {(time2 - time1) / game.history_len}")


if __name__ == "__main__":
    num_agents = 5

    processes = []
    for i in range(num_agents):
        process = Process(target=play_game, args=(i,))
        process.start()
        processes.append(process)

    for process in processes:
        process.join()


2024-02-23 13:04:16.961188: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5532 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-02-23 13:04:16.961468: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5532 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-02-23 13:04:16.969835: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5426 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-02-23 13:04:16.976530: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5360 MB memory:  -> device: 0, name: NVIDIA Ge

2: Game 0 finished in: 4.949431419372559. Moves: 17. Time per move: 0.291143024668974
4: Game 0 finished in: 32.49440622329712. Moves: 109. Time per move: 0.2981138185623589
2: Game 1 finished in: 37.50542640686035. Moves: 128. Time per move: 0.2930111438035965
0: Game 0 finished in: 51.58996343612671. Moves: 184. Time per move: 0.280380236065906
0: Game 1 finished in: 17.267571449279785. Moves: 55. Time per move: 0.31395584453235975
4: Game 1 finished in: 49.81297731399536. Moves: 139. Time per move: 0.3583667432661537
3: Game 0 finished in: 105.87225866317749. Moves: 358. Time per move: 0.29573256609826115
1: Game 0 finished in: 111.15045928955078. Moves: 423. Time per move: 0.2627670432377087
3: Game 1 finished in: 70.92088317871094. Moves: 190. Time per move: 0.3732678062037418
1: Game 1 finished in: 73.21766448020935. Moves: 263. Time per move: 0.27839416152170854


In [ ]:
from game import Game
import cppchess as chess
from multiprocessing import Process
from mcts import run_mcts
from config import Config
from time import time
from tf_funcs import fake_network
config = Config()

game = Game(chess.Board())
move, root = run_mcts(
    config,
    game,
    fake_network,
    None,
    False,
    300,
    30,
)
    





In [1]:
import numpy as np
neki = np.array([1, 2, 3, 4, 5], dtype=np.float32)
a = neki[3]
a

4.0

In [1]:
import actionspace.mapper as asm

In [2]:
asm.moves

array(['Q,N,1', 'Q,NE,1', 'Q,E,1', 'Q,SE,1', 'Q,S,1', 'Q,SW,1', 'Q,W,1',
       'Q,NW,1', 'Q,N,2', 'Q,NE,2', 'Q,E,2', 'Q,SE,2', 'Q,S,2', 'Q,SW,2',
       'Q,W,2', 'Q,NW,2', 'Q,N,3', 'Q,NE,3', 'Q,E,3', 'Q,SE,3', 'Q,S,3',
       'Q,SW,3', 'Q,W,3', 'Q,NW,3', 'Q,N,4', 'Q,NE,4', 'Q,E,4', 'Q,SE,4',
       'Q,S,4', 'Q,SW,4', 'Q,W,4', 'Q,NW,4', 'Q,N,5', 'Q,NE,5', 'Q,E,5',
       'Q,SE,5', 'Q,S,5', 'Q,SW,5', 'Q,W,5', 'Q,NW,5', 'Q,N,6', 'Q,NE,6',
       'Q,E,6', 'Q,SE,6', 'Q,S,6', 'Q,SW,6', 'Q,W,6', 'Q,NW,6', 'Q,N,7',
       'Q,NE,7', 'Q,E,7', 'Q,SE,7', 'Q,S,7', 'Q,SW,7', 'Q,W,7', 'Q,NW,7',
       'K,N,E', 'K,N,W', 'K,E,N', 'K,E,S', 'K,S,E', 'K,S,W', 'K,W,N',
       'K,W,S', 'P,N,R', 'P,N,B', 'P,N,K', 'P,NE,R', 'P,NE,B', 'P,NE,K',
       'P,NW,R', 'P,NW,B', 'P,NW,K'], dtype=object)

In [5]:
asm.action_space_indices[0, 0, :]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72])

In [8]:
asm.action_space[0:73]

array(['Q,N,1', 'Q,NE,1', 'Q,E,1', 'Q,SE,1', 'Q,S,1', 'Q,SW,1', 'Q,W,1',
       'Q,NW,1', 'Q,N,2', 'Q,NE,2', 'Q,E,2', 'Q,SE,2', 'Q,S,2', 'Q,SW,2',
       'Q,W,2', 'Q,NW,2', 'Q,N,3', 'Q,NE,3', 'Q,E,3', 'Q,SE,3', 'Q,S,3',
       'Q,SW,3', 'Q,W,3', 'Q,NW,3', 'Q,N,4', 'Q,NE,4', 'Q,E,4', 'Q,SE,4',
       'Q,S,4', 'Q,SW,4', 'Q,W,4', 'Q,NW,4', 'Q,N,5', 'Q,NE,5', 'Q,E,5',
       'Q,SE,5', 'Q,S,5', 'Q,SW,5', 'Q,W,5', 'Q,NW,5', 'Q,N,6', 'Q,NE,6',
       'Q,E,6', 'Q,SE,6', 'Q,S,6', 'Q,SW,6', 'Q,W,6', 'Q,NW,6', 'Q,N,7',
       'Q,NE,7', 'Q,E,7', 'Q,SE,7', 'Q,S,7', 'Q,SW,7', 'Q,W,7', 'Q,NW,7',
       'K,N,E', 'K,N,W', 'K,E,N', 'K,E,S', 'K,S,E', 'K,S,W', 'K,W,N',
       'K,W,S', 'P,N,R', 'P,N,B', 'P,N,K', 'P,NE,R', 'P,NE,B', 'P,NE,K',
       'P,NW,R', 'P,NW,B', 'P,NW,K'], dtype=object)